In [ ]:
import tracker3d
import pandas as pd
import numpy as np

In [ ]:
frame = pd.read_csv("datasets/standard_100MeV.csv")
train, target = tracker3d.loader.dataload(frame, nev=256, tpe=5, ts=4, npe=5, z_bounds=(0,0))
print(train)
print(target)
count = 0
for event in target:
    if np.count_nonzero(event) == 0:
        count = count + 1
print("The number of all 0 matrices:")
print("------", count, "------")

In [ ]:
from keras.layers import Dense, LSTM, Dropout, GRU
from keras.models import Sequential
import keras

In [ ]:
input_shape = train[0].shape # Shape of an event.
output_shape = len(target[0][0]) # Number of tracks per event

batch_size = 32
epochs     = 512
valsplit   = 0.25
opt        = 'rmsprop' # optimizer
trackerModel = Sequential()
trackerModel.add(GRU(32, return_sequences=True, input_shape=input_shape, dropout=.3, recurrent_dropout=.2))
trackerModel.add(Dense(output_shape, activation='softmax'))

trackerModel.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
#tracker.model.summary()

In [ ]:
modelpath = 'simple.h5'
hist = trackerModel.fit(train, target, epochs=epochs, batch_size=batch_size,
                         verbose=0, validation_split=valsplit,
                         callbacks=[keras.callbacks.ModelCheckpoint(filepath=modelpath, verbose=0)])
print("Ding! All done.")


In [ ]:
score, acc = trackerModel.evaluate(train, target, batch_size=batch_size)
print("\nTest Score:    {}".format(score))
print("Test Accuracy: {}".format(acc))
tracker3d.utils.graph_losses([("Categorical Cross Entropy", hist)])

In [ ]:
predictions = trackerModel.predict(train[:len(train)], batch_size=batch_size)
count = 0
#for i, outMax in enumerate(predictions[-50:]):
#    tracker3d.utils.print_metrics(train[i], target[i], outMax)
#    if(tracker3d.utils.print_metrics(train[i], target[i], outMax, verbose=False)):
#        print("========{}========".format(i))
#        tracker3d.utils.plot2D(train[i], tracker3d.utils.from_categorical(target[i]))
#        tracker3d.utils.plot2D(train[i], tracker3d.utils.from_categorical(predictions[i]))
tracker3d.utils.print_metrics(train[8], target[8], predictions[8])
tracker3d.utils.plot2D(train[8], tracker3d.utils.from_categorical(target[8]))
tracker3d.utils.plot2D(train[8], tracker3d.utils.from_categorical(predictions[8]))
#tracker3d.utils.display_side_by_side(target[8:9], predictions[8:9])
print(train[8])
print(target[8])
print(predictions[8])
for i, outMax in enumerate(predictions):
    if(tracker3d.utils.print_metrics(train[i], target[i], outMax, verbose=False)==False):
        count = count + 1
        
print("The total number of completely correct events:", count)
#        df = [pd.DataFrame(data=discreteOut), pd.DataFrame(data=target[i])]
#        multi_column_df_display(df)